<h2>Scrape and Imports</h2>

In [27]:
# All the imports

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

import requests
from pandas.io.json import json_normalize

import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!pip install folium==0.5.0
import folium 

Montreal raw Data

In [28]:
# The code was removed by Watson Studio for sharing.

In [29]:

body = client_a0176b9ee52d41f7bf60b67d1fc261ec.get_object(Bucket='capstone-donotdelete-pr-tn7aynsygnvldo',Key='quartierreferencehabitation.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
df_data_3.head()


,no_qr,Borough,no_arr,Neighbourhood,nom_mun
0,53,Rivière-des-Prairies,33.0,Rivière-des-Prairies–Pointe-aux-Trembles,Montréal
1,24,Beaurivage,22.0,Mercier–Hochelaga-Maisonneuve,Montréal
2,23,Tétreaultville,22.0,Mercier–Hochelaga-Maisonneuve,Montréal
3,22,Dupéré,22.0,Mercier–Hochelaga-Maisonneuve,Montréal
4,25,Guybourg,22.0,Mercier–Hochelaga-Maisonneuve,Montréal


In [30]:
mount = df_data_3.drop(columns=['no_qr', 'nom_mun','no_arr']).rename(columns={"Neighbourhood":"Borough","Borough":"Neighbourhood"})
mount.shape

(91, 2)

In [31]:

body = client_a0176b9ee52d41f7bf60b67d1fc261ec.get_object(Bucket='capstone-donotdelete-pr-tn7aynsygnvldo',Key='Neighbourhoods.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_5 = pd.read_csv(body)
df_data_5.head()


,Neighbourhood,Latitude,Longitude
0,Rivière-des-Prairies,45.6677,-73.5462
1,Beaurivage,45.5982,-73.5139
2,Tétreaultville,45.6018,-73.5246
3,Dupéré,45.6046,-73.5357
4,Guybourg,45.5742,-73.5206


Time to merge!

In [32]:
# Combine the two data sets
city = mount.merge(df_data_5, how='inner', on='Neighbourhood')
city.head()

,Neighbourhood,Borough,Latitude,Longitude
0,Rivière-des-Prairies,Rivière-des-Prairies–Pointe-aux-Trembles,45.6677,-73.5462
1,Beaurivage,Mercier–Hochelaga-Maisonneuve,45.5982,-73.5139
2,Tétreaultville,Mercier–Hochelaga-Maisonneuve,45.6018,-73.5246
3,Dupéré,Mercier–Hochelaga-Maisonneuve,45.6046,-73.5357
4,Guybourg,Mercier–Hochelaga-Maisonneuve,45.5742,-73.5206


In [33]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(city['Borough'].unique()),
        city.shape[0]
    )
)

The dataframe has 20 boroughs and 91 neighbourhoods.


In [34]:
address = 'Montreal, QC'

geolocator = Nominatim(user_agent="qb_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Montreal are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Montreal are 45.4972159, -73.6103642.


In [35]:
# create map of Quebec City using latitude and longitude values
map_quebec = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(city['Latitude'], city['Longitude'], city['Borough'], city['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_quebec)  
    
map_quebec

<h2>Finished with the Data. Setting up the maps!</h2>

In [36]:
# Foursqaure ID

import json
{
    "tags": [
        "remove-input",
    ]
}
CLIENT_ID = '4F2TWL40HEUEKJXU01ZUMRE4ZNYOGGTRNMPLGRVNJT33J1KJ' # your Foursquare ID
CLIENT_SECRET = 'O5MCRNOS2HOODSSYWX15PYEVB05WZMPY2VGB3GC1K4KQBCFG' # your Foursquare Secret
ACCESS_TOKEN = 'KFUMX2PWRS3FVCUZRGFCYPKLJ1VEKWDLC2NWLHOW24M3S5C4'
version = '20200605' # Foursquare API version
limit = 100 # A default Foursquare API limit value
radius = 1000 

First Neighbourhood

Create Foursquare link

In [43]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        categoryId = '4d4b7105d754a06374d81259' # ID of 'Food'
        url = 'https://api.foursquare.com/v2/venues/search?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            categoryId, 
            CLIENT_ID, 
            CLIENT_SECRET, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
city_venues = getNearbyVenues(names=city['Neighbourhood'],
                                   latitudes=city['Latitude'],
                                   longitudes=city['Longitude']
                                  )

Rivière-des-Prairies


KeyError: 'venues'

In [ ]:
city_venues.head()

In [ ]:
print('There are {} uniques categories.'.format(len(city_venues['Venue Category'].unique())))

<h2> Now let's look at what's in Montreal</h2>

In [ ]:
# one hot encoding
city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
city_onehot['Neighbourhood'] = city_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()

In [ ]:
city_grouped = city_onehot.groupby('Neighbourhood').mean().reset_index()
city_grouped.head()

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = city_grouped['Neighbourhood']

for ind in np.arange(city_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

<h2>Make a map!</h2>

In [40]:
kclusters = 5

city_grouped_clustering = city_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

NameError: name 'city_grouped' is not defined

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

city_merged = city

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
city_merged = city_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

city_merged.tail() # check the last columns!

In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_merged['Latitude'], city_merged['Longitude'], city_merged['Neighbourhood'], city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'kclusters' is not defined

<h2>Chloropleth Map</h2>

In [ ]:

# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(df_can['Count'].min(),
                              df_can['Count'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist()
threshold_scale[-1] = threshold_scale[-1] + 1 

latitude = 37.77
longitude = -122.42

cali = folium.Map(location=[latitude, longitude], zoom_start=12)

cali.choropleth(
    geo_data=world_geo,
    data=df_can,
    columns=['Neighborhood', 'Count'],
    key_on='feature.properties.DISTRICT',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Crime in SanFrancisco'
)

# display map
cali